In [105]:
# Various setup, probably mostly unused

from env import TestEnv
import pandas as pd
import json
import os
from conf import LisaLogging
from bart.common.Utils import area_under_curve
from trappy.plotter import plot_trace
from IPython.display import display
from trappy import ILinePlot
from trappy.stats.grammar import Parser
import pandas as pd
from trace import Trace
LisaLogging.setup()
import logging
logging.getLogger('Trace').setLevel(logging.ERROR)
logging.getLogger('Analysis').setLevel(logging.WARNING)
%matplotlib inline
from platforms.juno_energy import juno_energy
from platforms.pixel_energy import pixel_energy
import tests.eas.generic
from tests.eas.generic import EnergyModelTest
import numpy as np
from IPython.display import display
from wlgen import RTA, Periodic

2017-02-03 15:37:09,101 INFO    : root         : Using LISA logging configuration:
2017-02-03 15:37:09,102 INFO    : root         :   /home/brejac01/sources/lisa/logging.conf


In [2]:
# Some nonsense to get caiman to work on Brendan's computer
p = os.getenv('PATH').split(':')
caiman_path = '/opt/ds5_v5.23.0/bin'
if caiman_path not in p:
    p.insert(0, caiman_path)
    os.environ['PATH'] = ':'.join(p)

In [3]:
from platforms.juno_energy import juno_energy

In [4]:
from platforms.hikey_energy import hikey_energy

In [5]:
def get_nrg(model, util):
    nrg = model.estimate_from_cpu_util(util, zero_idle=True)
    return sum(nrg.values())

In [6]:
def spread_evenly(ntasks, ncpus):
    ret = [0 for _ in range(ncpus)]
    for i in range(ntasks):
        ret[i % ncpus] += 1
    return ret

def get_various_packing_placements(ntasks, ncpus):
    max_spread = spread_evenly(ntasks, ncpus)
    
    max_pack = [ntasks] + [0] * (ncpus - 1)
    
    half_pack = [ntasks / 2] + spread_evenly(ntasks - (ntasks / 2), ncpus -1)
    
    return [max_spread, half_pack, max_pack]

In [110]:
te = TestEnv(test_conf={'modules': ['cgroups'], 'ftrace': {'events': ['sched_switch']}}, force_new=True)

2017-02-03 15:39:47,864 INFO    : TestEnv      : Using base path: /home/brejac01/sources/lisa
2017-02-03 15:39:47,866 INFO    : TestEnv      : Loading default (file) target configuration
2017-02-03 15:39:47,867 INFO    : TestEnv      : Loading target configuration [/home/brejac01/sources/lisa/target.config]...
2017-02-03 15:39:47,871 INFO    : TestEnv      : Loading custom (inline) test configuration
2017-02-03 15:39:47,872 INFO    : TestEnv      : External tools using:
2017-02-03 15:39:47,873 INFO    : TestEnv      :    ANDROID_HOME: /work/android-sdk-linux
2017-02-03 15:39:47,874 INFO    : TestEnv      :    CATAPULT_HOME: /home/brejac01/sources/lisa/tools/catapult
2017-02-03 15:39:47,876 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cgroups', 'cpufreq']
2017-02-03 15:39:47,877 INFO    : TestEnv      : Connecting Android target [04D314DF000BC57F]
2017-02-03 15:39:47,878 INFO    : TestEnv      : Connection settings:
2017-02-03 15:39:47,879 INFO    : TestEnv      :    {'

In [ ]:
def get_everything(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    task_distribs = get_various_packing_placements(tasks_per_cpu * ncpus, ncpus)
    
    util_distribs = []
    wloads = []
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)
        
    return util_distribs

    wload = RTA(te.target, 'periodic_{}x_{}pct', te.calibration())
    

    for task_distrib in task_distr

In [96]:
def get_task_distribs(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    task_distribs = get_various_packing_placements(tasks_per_cpu * ncpus, ncpus)
    return task_distribs

In [101]:
def get_util_distribs(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    util_distribs = []
    
    task_cap = int(1024 * (task_pct / 100.))
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)
    return util_distribs

In [ ]:
def get_wloads(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    for task_distrib in 

In [102]:
util_distribs = get_util_distribs(hikey_energy)
for ud in util_distribs:
    display(hikey_energy.estimate_from_cpu_util(ud))

{(0,): 63.53932584269663,
 (0, 1, 2, 3): 25.20224719101124,
 (1,): 63.53932584269663,
 (2,): 63.53932584269663,
 (3,): 63.53932584269663,
 (4,): 0.0,
 (4, 5, 6, 7): 0.0,
 (5,): 0.0,
 (6,): 0.0,
 (7,): 0.0}

{(0,): 110.39295392953929,
 (0, 1, 2, 3): 39.35772357723577,
 (1,): 50.77235772357723,
 (2,): 50.77235772357723,
 (3,): 38.84823848238482,
 (4,): 0.0,
 (4, 5, 6, 7): 0.0,
 (5,): 0.0,
 (6,): 0.0,
 (7,): 0.0}

{(0,): 290.06715506715506,
 (0, 1, 2, 3): 81.993894993895,
 (1,): 0.0,
 (2,): 0.0,
 (3,): 0.0,
 (4,): 0.0,
 (4, 5, 6, 7): 0.0,
 (5,): 0.0,
 (6,): 0.0,
 (7,): 0.0}

In [8]:
hikey_energy.cpu_nodes[0].min_capacity

178